# IMPORTS

In [2]:
import itertools
import sys
import time

# FUNCTION DEFINITIONS

Class Definitions

In [3]:
class Node(object):
    def __init__(self, data=None, next_node=None):
        self.data = data
        self.next_node = next_node
        
    def get_data(self):
        return self.data
    
    def get_next(self):
        return self.next_node
    
    def set_next(self, new_next):
        self.next_node = new_next

    #https://www.codefellows.org/blog/implementing-a-singly-linked-list-in-python/

    def get_state(self):
        return self.data[1]

    def get_direction(self):
        return self.data[3]

    def get_successor1(self):
        successor = itertools.combinations(self.state, len(self.state)-2)
        successor_pairs = []
        for i in successor:
            left_group = self.state[:]
            for j in i:
                if j in self.state:
                    left_group.remove(j)
            successor_pairs.append((list(i), left_group))
        return successor_pairs

    # TODO: try to refactor
    def get_successor2(self, crossed):
        successor = []
        for i in crossed:
            with_returnee = self.state[:]
            with_returnee.append(i)
            with_returnee.sort()
            successor.append((with_returnee, i))

    def get_crossed_list(self, original):
        original_state = original[:]
        state = self.state[:]
        for i in original_state[:]:
            if i in state:
                original_state.remove(i)
                state.remove(i)
        crossed = original_state[:]
        return crossed

Administrative functions

In [4]:
def print_time(start, end, start2, end2):
    runtime = end-start
    wallruntime = end2-start2
    print(runtime)
    print(wallruntime)
    return

In [5]:
def print_fringe (fringe_parameter):
    printedlist = []
    for i in fringe_parameter:
        printedlist.append(i.data)
    for i in printedlist:
        print (i)
    return

In [6]:
def print_path(node_parameter):
    current = node_parameter
    path = []
    while current.next_node != None:
        path.append(current.data[1])
        current=current.next_node
    path.append(current.data[1])
    path.reverse()
    for i in path:
        print(i)
    return

In [7]:
def print_crossing(node_parameter):
    current = node_parameter
    output = []
    previous_state = []
    while current.next_node != None:
        state = current.data[1]
        direction = current.data[3]
        
        if direction == 0:
            crossed = get_crossed_list(state, previous_state)
            crossed.append('<-')
            output.append(crossed)
        else:
            crossed = get_crossed_list(previous_state, state)
            crossed.append('->')
            output.append(crossed)
        
        previous_state = current.data[1]
        current = current.next_node
        
    state = current.data[1]
    direction = current.data[3]

    crossed = get_crossed_list(previous_state, state)
    if direction == 0:
        crossed.append('<-')
        output.append(crossed)
    else:
        crossed.append('->')
        output.append(crossed)
    
    output.reverse()
    for i in range(len(output)-1):
        for j in output[i]:
            print(j, end=' ')
        print()
    
    return

Calculation functions

In [8]:
def crossings(n_parameter):
    if n_parameter == 0: return 0
    elif n_parameter == 1: return 0
    elif n_parameter == 2: return 0
    if n_parameter == 3: return 3
    else:
        times = crossings(n_parameter-1)+2
        return times

Comparison functions

In [9]:
def max_of(triple_parameter):
    tup1 = int(triple_parameter[1][0])
    tup2 = int(triple_parameter[1][1])
    if tup2 > tup1: return tup2
    else: return tup1

Fringe operations

In [10]:
def insert(node_parameter, subfringe_parameter):
    for i in subfringe_parameter:
        if i.data[1] == node_parameter.data[1]: return
    subfringe_parameter.append(node_parameter)
    return

In [11]:
def remove(fringe_parameter):
    for a in range(len(fringe_parameter)):
        if subfringe_empty(fringe_parameter, a): continue
        #sort_fringe(fringe_parameter[a])
        node_parameter = fringe_parameter[a][0]
        fringe_parameter[a].remove(fringe_parameter[a][0])
        return node_parameter

In [12]:
def remove_duplicate(node_parameter, subfringe_parameter):
    new_sub = []
    for i in range(len(subfringe_parameter)):
        if subfringe_parameter[i].data[1] != node_parameter.data[1]:
            #subfringe_parameter[1].remove(subfringe_parameter[i])
            new_sub.append(subfringe_parameter[i])
        elif subfringe_parameter[i].data[3] != node_parameter.data[3]:
            new_sub.append(subfringe_parameter[i])
    return new_sub

In [13]:
def fringe_empty(fringe_parameter):
    for a in fringe_parameter:
        if not(len(a)==0):
            return False
        else: continue
    return True

In [14]:
def subfringe_empty(fringe_parameter, subfringe_index):
    if len(fringe_parameter[subfringe_index]) == 0: return True
    else: return False

Table operations

In [20]:
def in_table(node_parameter, table_parameter):
    if table_parameter[tuple(node_parameter.data[1])] != 999999999: return (1, table_parameter[tuple(node_parameter.data[1])])
    else: return (0, 999999999)

## Implementation

To represent the problem, the implementation takes into account three aspects of the problem: **the states, the transitions, and the goal.**
<br><br>
*States*
> The states are the people left who need to cross the bridge to get to the other side. In the program implementation, this is represented by a list of integers that represent the amount of time it takes for them to cross. For this report, we will refer to the side from which people need to cross as the left side, and the side where people need to go as the right side. 

*Transition/successor functions*
> The states reach other states through transition/successor functions. The implementation of the problem uses two functions: one for when two people cross to the other side and another for when one person comes back with the flashlight. This means that the initial node can go to states which are two elements shorter than it, and these states in turn can go to states which are one element greater, and so on (e.g. ABCD -> AB -> ABC -> A).

*Goal*
> The goal is reached when there are no more people on the left side. This is represented by states with no elements which is equivalent to an empty list for a state.

<br><br><br>
The major elements of the program's implementation of the problem include: *the node, the fringe and hash tables, and successor functions*
<br><br>
*Node*<br>
The node is the enclosing unit that represents a state and the details associated with it. It has both data and a pointer to another node. <br>
The data contained within the node include:  <br>
   > 1) an *integer cost* of the path from the initial node to that state, <br>
   > 2) a *list of integers* for the time it takes each person to cross,<br>
   > 3) the *number of people on the left side* which is the length of the list, <br>
   > 4) the *direction* of crossing, and <br>
   > 5) another integer cost of the *actual cost* from the initial node to that state.<br>
    
<div class="alert alert-block alert-info">The first element in the data is the actual cost in UCS and the actual cost plus the heuristic value for the A star search. The fifth element is only existent in the A star search to store the actual cost.
The pointer points to the predecessor of the state which is used when backtracking to the initial node to form a path when a solution is found.</div>

*Fringe*<br>
> The fringe is represented by a list of lists. Its length is equal to the highest cost possible which occurs when the person with the maximum cost crosses the bridge every time. The elements of the sub-lists are the nodes which are queued for exploration/expansion. The sub-lists are indexed according to the path cost of the member nodes (i.e. all states with the same path cost belong to the same sub-list). Removing an element for expansion involves iterating over the main list checking if the sub-list in that index is empty, if not, then the first element is returned. Inserting a node to the fringe is done simply by appending the node to the sub-list indexed by the node's path cost. This implementation was chosen to reduce the overhead time cost of sorting the fringe every time a node is inserted.

*Hash tables*
> The hash tables, implemented through dictionaries, are used to keep track of states in the fringe set and in the explored set. The keys are all the possible combinations which can be taken from the set of people. Its value is initially set to a large number and is updated every time a node is explored or added to the fringe. For example, before a node is added, the table is checked if a node with the same state already exists in the fringe or in the explored set. If the value in the table is the initial large number, then the state is not yet in the fringe and is simply added to it. If the value in the table is not the initial large number, the path cost of the node is compared with that value. If the path cost is smaller, then the program adds the node to the appropriate sub-list in the fringe and removes the node in the sub-list indexed by the value in the table.

*Successor functions*

>The first successor function generates the possible combinations of length k-2, where k is the number of people on the left side. This is done through the combination method in an imported module called itertools. The function returns tuples that include the combinations of people on the left side and the two people who crossed.

>The second successor function first determines the list of people in the right (initial state minus current state). l number of new lists are formed, where l is the number of people on the right side. Each member of the right side is added to the left side to form a list. The function returns tuples that include the new state and the person that came back.
The implementation of the Uniform Cost Search and the A star Search are identical except for the evaluation functions of the two and the structure of the node wherein A star needs one more value to store the actual cost along with the path cost. The general flow of the algorithm is as follows:
1. The start state/node is initialized based on the user input<br>
2. An early check for trivial cases is made:
    If the length of the input is 0, then that means that there are no people to consider. The program simply terminates.<br>
    Else, if the length of the input is 1, then one crossing is all it takes and the total cost is that of the lone element.<br>
    Else, if the length of the input is 2, also one crossing is sufficient and the total cost is the larger of the two elements.<br>
3. The start node is inserted into the fringe and the corresponding values for the tables are updated<br>
4. This loop which is done until a solution is found:<br>
	a.)  If the fringe is empty, then a failure is returned.<br>
    b.)  Else, the highest priority (one with least cost) in the fringe is taken out. The tables are updated.<br>
    c.)  The state and the direction of crossing in the data is extracted.<br>
    d.)  The state is checked if it is a goal state. If yes, the search terminates and outputs solution.<br>
    e.)  Else, the direction is checked to choose the right successor function<br>
    f.)  If the direction is to cross (1), then the first successor function is chosen.<br>
	g.)  Else, if the direction is to come back (0), then the second is chosen.<br>
	h.)  The successors of the current state are generated.<br>
    i.)  For every successor states, the pathcost is calculated (UCS: pathcost = actual cost, Astar: pathcost = actual cost + heuristic value). A node is created and its data field is populated).<br>
    j.)  The tables are checked to test for membership of the node in the fringe or explored set. If the node is not in the fringe or the explored set, it is appended to the fringe.<br>
    k.)  If the node is in the fringe, the pathcosts are compared. If the pathcost of the node is lower, the one in the fringe is removed and the node is added.<br>

### A * Heuristic
The heuristic we chose is the maximum possible cost moving from that state to any other state other. Simply, the heuristic is the person with the highest cost in that state. It is said that function is admissible if h(n) ≤ h*(n), where h*(n) is the true cost and h(n) is the chosen heuristic. Proof is written down below.
        
    For cases wherein two or more people have yet to arrive at the right side:
      The value of h(n) is the maximum cost between the members of the group. We have yet to factor in the cost of moving the other members of the group into the value which is the true cost [h*(n)] from that particular state to the goal state. From this, we can safely conclude that h(n) < h*(n).
            
    For cases wherein only one person has yet to arrive at the right side:
        The value of h(n) is still the maximum cost between the members of the group. Since there is only one member in the group left, the maximum cost is also the only remaining cost. We do not have to factor in any other members of the group and therefore the heuristic now represents the true cost from that state to the goal state. From this we can safely conclude that h(n) = h*(n).
          
Since it is proven that h(n) ≤ h*(n), we can conclude that the heuristic is admissible.

In [24]:
def search(initial, choice):
    start = time.perf_counter()
    wallstart = time.time()

    n = len(initial)

    if choice == 0:
        
        # Trivial cases check
        
        if initial[0]=='':
            print ("No people to cross.")
            sys.exit(1)
        elif n == 1:
            print (int(initial[0]), 2)
            print (int(initial[0]), '->')
            sys.exit(1)
        elif n==2:
            print (max(int(initial[0]), int(initial[1])), 2)
            print (int(initial[0]), int(initial[1]), '->')
            sys.exit(1)
     
        # Start node initialization

        # In[44]:

        for i in range(len(initial)):
            initial[i]=int(initial[i])
        initial.sort()
        required_crossings = crossings(n)

        initial_node_data = (0, initial, len(initial), 1)
        initial_node = Node(initial_node_data)


        # Container for the fringe, explored states, solution nodes, and tables initialization

        # In[45]:

        maximum = 0
        for i in initial:
            if int(i) > maximum: maximum = int(i)

        maximum = maximum*required_crossings+1

        fringe = []
        explored = []
        for i in range(maximum):
            fringe.append([])
            explored.append([])

        dicstatelist0 = {}
        dicstatelist1 = {}
        dicexstatelist0 = {}
        dicexstatelist1 = {}

        m = n
        while m>=0:
            combi = itertools.combinations(initial, m)
            for i in combi:
                dicstatelist0[i] = 999999999
                dicstatelist1[i] = 999999999
                dicexstatelist0[i] = 999999999
                dicexstatelist1[i] = 999999999
            m-=1

        solutions = []


        # # UNIFORM COST SEARCH

        # 1) Insert initial state to fringe

        # In[46]:

        insert(initial_node, fringe[initial_node.data[0]])
        dicstatelist0[tuple(initial_node.data[1])] = 0


        # 2) Repeat

        # In[47]:

        while(True):

            ### RETURN FAILURE IF FRINGE IS EMPTY
            if (fringe_empty(fringe)):
                sys.exit(0)

            ### REMOVE FROM FRINGE THE NODE WITH HIGHEST PRIORITY
            node = remove(fringe)
            explored.append(node)

            ### GET THE STATE FROM NODE
            state = get_state(node)
            direction = get_direction(node)

            ### IF THE STATE IS THE GOAL, RETURN SOLUTION
            if len(state) == 0:
                solutions.append(node)

                print('Input: ', end=' ')
                for i in initial:
                    print(i, end=' ')
                print()
                print('Output: ', solutions[0].data[0], len(explored))        
                print_crossing(solutions[0])
                #print_path(solutions[0])

                end = time.perf_counter()
                wallend = time.time()
                print_time(start, end, wallstart, wallend)
                sys.exit(1)

            cond1 = (1, 1)
            cond2 = (0, 0)

            ### GET THE SUCCESSORS OF THE STATE
            ### SUCCESSOR FUNCTION FOR TWO CROSSING PEOPLE
            if direction == 1:

                dicexstatelist1[tuple(node.data[1])] = node.data[0]
                dicstatelist1[tuple(node.data[1])] = 999999999

                successors = get_successor1(state, len(state)-2)

                ### CREATE A NODE FOR EVERY SUCCESSOR
                for k in successors:
                    pathcost = node.data[0]+int(max_of(k))
                    nodeprime = Node((pathcost, k[0], len(k[0]), 0))
                    nodeprime.set_next(node)

                    cond2 = in_table(nodeprime, dicexstatelist0)
                    cond1 = in_table(nodeprime, dicstatelist0)

                    ### IF THE SUCCESSOR IS NOT IN FRINGE OR EXPLORED, INSERT NODE TO FRINGE
                    if cond1[0]!=1 and cond2[0]!=1:
                        insert(nodeprime, fringe[nodeprime.data[0]])
                        dicstatelist0[tuple(nodeprime.data[1])] = nodeprime.data[0]

                    ### ELSE IF THE SUCCESSOR IS IN THE FRINGE,
                    ### REPLACE THE ONE IN THE FRINGE IF THE COST OF GOING TO THIS SUCCESSOR IS LOWER
                    elif cond1[0]==1:
                        if nodeprime.data[0]<cond1[1]:
                            insert(nodeprime, fringe[nodeprime.data[0]])
                            fringe[cond1[1]] = remove_duplicate(nodeprime, fringe[cond1[1]])
                            if dicstatelist0[tuple(nodeprime.data[1])] > nodeprime.data[0]:
                                dicstatelist0[tuple(nodeprime.data[1])] = nodeprime.data[0]

            ### SUCCESSOR FUNCTION FOR ONE PERSON CROSSING BACK           
            elif direction == 0:

                dicexstatelist0[tuple(node.data[1])] = node.data[0]
                dicstatelist0[tuple(node.data[1])] = 999999999

                crossed = get_crossed_list(state, initial_node.data[1])
                successors = get_successor2(state, crossed)

                ### CREATE A NODE FOR EVERY SUCCESSOR
                for k in successors:
                    pathcost = node.data[0]+int(k[1])
                    nodeprime = Node((pathcost, k[0], len(k[0]), 1))
                    nodeprime.set_next(node)

                    cond2 = in_table(nodeprime, dicexstatelist1)
                    cond1 = in_table(nodeprime, dicstatelist1)

                    ### IF THE SUCCESSOR IS NOT IN FRINGE OR EXPLORED, INSERT NODE TO FRINGE
                    if cond1[0]!=1 and cond2[0]!=1:
                        insert(nodeprime, fringe[nodeprime.data[0]])
                        dicstatelist1[tuple(nodeprime.data[1])] = nodeprime.data[0]

                    ### ELSE IF THE SUCCESSOR IS IN THE FRINGE,
                    ### REPLACE THE ONE IN THE FRINGE IF THE COST OF GOING TO THE SUCCESSOR IS LOWER
                    elif cond1[0]==1:
                        if nodeprime.data[0]<cond1[1]:
                            insert(nodeprime, fringe[nodeprime.data[0]])
                            fringe[cond1[1]] = remove_duplicate(nodeprime, fringe[cond1[1]])
                            if dicstatelist1[tuple(nodeprime.data[1])] > nodeprime.data[0]:
                                dicstatelist1[tuple(nodeprime.data[1])] = nodeprime.data[0]
            
    elif choice == 1: 
        # Trivial cases check

        # In[21]:

        if initial[0]=='':
            print ("No people to cross.")
            sys.exit(1)
        elif n == 1:
            print (int(initial[0]), 2)
            print (int(initial[0]), '->')
            sys.exit(1)
        elif n==2:
            print (max(int(initial[0]), int(initial[1])), 2)
            print (int(initial[0]), int(initial[1]), '->')
            sys.exit(1)


        # Start node initialization

        # In[22]:

        for i in range(len(initial)):
            initial[i]=int(initial[i])
        initial.sort()
        required_crossings = crossings(n)

        initial_node_data = (0, initial, len(initial), 1, 0)
        initial_node = Node(initial_node_data)


        # Container for the fringe, explored states, solution nodes, and tables initialization

        # In[23]:

        maximum = 0
        for i in initial:
            if int(i) > maximum: maximum = int(i)

        maximum = maximum*required_crossings+1

        fringe = []
        explored = []
        for i in range(maximum):
            fringe.append([])
            explored.append([])

        dicstatelist0 = {}
        dicstatelist1 = {}
        dicexstatelist0 = {}
        dicexstatelist1 = {}

        m = n
        while m>=0:
            combi = itertools.combinations(initial, m)
            for i in combi:
                dicstatelist0[i] = 999999999
                dicstatelist1[i] = 999999999
                dicexstatelist0[i] = 999999999
                dicexstatelist1[i] = 999999999
            m-=1

        solutions = []


        # # A* SEARCH

        # 1) Insert initial state to fringe

        # In[24]:

        insert(initial_node, fringe[initial_node.data[0]])
        dicstatelist0[tuple(initial_node.data[1])] = 0


        # 2) Repeat

        # In[25]:

        while(True):

            ### RETURN FAILURE IF FRINGE IS EMPTY
            if (fringe_empty(fringe)):
                sys.exit(0)

            ### REMOVE FROM FRINGE THE NODE WITH HIGHEST PRIORITY
            node = remove(fringe)
            explored.append(node)

            ### GET THE STATE FROM NODE
            state = get_state(node)
            direction = get_direction(node)

            ### IF THE STATE IS THE GOAL, RETURN SOLUTION
            if len(state) == 0:
                solutions.append(node)

                print('Input: ', end=' ')
                for i in initial:
                    print(i, end=' ')
                print()
                print('Output: ', solutions[0].data[0], len(explored))        
                print_crossing(solutions[0])
                #print_path(solutions[0])

                end = time.perf_counter()
                wallend = time.time()
                print_time(start, end, wallstart, wallend)
                sys.exit(1)

            cond1 = (1, 1)
            cond2 = (0, 0)

            ### GET THE SUCCESSORS OF THE STATE
            ### SUCCESSOR FUNCTION FOR TWO CROSSING PEOPLE
            if direction == 1:

                dicexstatelist1[tuple(node.data[1])] = node.data[0]
                dicstatelist1[tuple(node.data[1])] = 999999999

                successors = get_successor1(state, len(state)-2)

                ### CREATE A NODE FOR EVERY SUCCESSOR
                for k in successors:
                    actual_cost = node.data[4] + int(max_of(k))
                    heuristic_value = 0
                    if len(k[0]) > 0: heuristic_value = max(k[0])
                    pathcost = actual_cost + heuristic_value
                    nodeprime = Node((pathcost, k[0], len(k[0]), 0, actual_cost))
                    nodeprime.set_next(node)

                    cond2 = in_table(nodeprime, dicexstatelist0)
                    cond1 = in_table(nodeprime, dicstatelist0)

                    ### IF THE SUCCESSOR IS NOT IN FRINGE OR EXPLORED, INSERT NODE TO FRINGE
                    if cond1[0]!=1 and cond2[0]!=1:
                        insert(nodeprime, fringe[nodeprime.data[0]])
                        dicstatelist0[tuple(nodeprime.data[1])] = nodeprime.data[0]

                    ### ELSE IF THE SUCCESSOR IS IN THE FRINGE,
                    ### REPLACE THE ONE IN THE FRINGE IF THE COST OF GOING TO THIS SUCCESSOR IS LOWER
                    elif cond1[0]==1:
                        if nodeprime.data[0]<cond1[1]:
                            insert(nodeprime, fringe[nodeprime.data[0]])
                            fringe[cond1[1]] = remove_duplicate(nodeprime, fringe[cond1[1]])
                            if dicstatelist0[tuple(nodeprime.data[1])] > nodeprime.data[0]:
                                dicstatelist0[tuple(nodeprime.data[1])] = nodeprime.data[0]      

            ### SUCCESSOR FUNCTION FOR ONE PERSON CROSSING BACK            
            elif direction == 0:

                dicexstatelist0[tuple(node.data[1])] = node.data[0]
                dicstatelist0[tuple(node.data[1])] = 999999999

                crossed = get_crossed_list(state, initial_node.data[1])
                successors = get_successor2(state, crossed)

                ### CREATE A NODE FOR EVERY SUCCESSOR
                for k in successors:
                    actual_cost = node.data[4] + int(k[1])
                    heuristic_value = 0
                    if len(k[0]) > 0: heuristic_value = max(k[0])
                    pathcost = actual_cost + heuristic_value
                    nodeprime = Node((pathcost, k[0], len(k[0]), 1, actual_cost))
                    nodeprime.set_next(node)

                    cond2 = in_table(nodeprime, dicexstatelist1)
                    cond1 = in_table(nodeprime, dicstatelist1)

                    ### IF THE SUCCESSOR IS NOT IN FRINGE OR EXPLORED, INSERT NODE TO FRINGE
                    if cond1[0]!=1 and cond2[0]!=1:
                        insert(nodeprime, fringe[nodeprime.data[0]])
                        dicstatelist1[tuple(nodeprime.data[1])] = nodeprime.data[0]

                    ### ELSE IF THE SUCCESSOR IS IN THE FRINGE,
                    ### REPLACE THE ONE IN THE FRINGE IF THE COST OF GOING TO THIS SUCCESSOR IS LOWER
                    elif cond1[0]==1:
                        if nodeprime.data[0]<cond1[1]:
                            insert(nodeprime, fringe[nodeprime.data[0]])
                            fringe[cond1[1]] = remove_duplicate(nodeprime, fringe[cond1[1]])
                            if dicstatelist1[tuple(nodeprime.data[1])] > nodeprime.data[0]:
                                dicstatelist1[tuple(nodeprime.data[1])] = nodeprime.data[0]






In [25]:
#UCS

search([1,2,5,10],0)

Input:  1 2 5 10 
Output:  17 73
1 2 -> 
1 <- 
5 10 -> 
2 <- 
1 2 -> 
0.0004653999931178987
0.0010020732879638672


SystemExit: 1

c:\Users\nicko\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3450: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [26]:
#UCS

search([1,2,5,10],1)

Input:  1 2 5 10 
Output:  17 66
1 2 -> 
1 <- 
5 10 -> 
2 <- 
1 2 -> 
0.001269000000320375
0.0010020732879638672


SystemExit: 1

In [29]:
#UCS

search([1,2,5,10,3,4,14,18,20,50],0)

Input:  1 2 3 4 5 10 14 18 20 50 
Output:  104 2876
1 2 -> 
1 <- 
5 10 -> 
2 <- 
1 2 -> 
1 <- 
14 18 -> 
2 <- 
1 2 -> 
1 <- 
20 50 -> 
2 <- 
1 2 -> 
1 <- 
1 3 -> 
1 <- 
1 4 -> 
0.4797277000034228
0.4800689220428467


SystemExit: 1

In [62]:
#A*

search([1,2,5,10,3,4,14,18,20,50],1)

Input:  1 2 3 4 5 10 14 18 20 50 
Output:  104 2743
1 2 -> 
1 <- 
5 10 -> 
2 <- 
1 2 -> 
1 <- 
20 50 -> 
2 <- 
1 2 -> 
1 <- 
1 3 -> 
1 <- 
1 4 -> 
1 <- 
14 18 -> 
2 <- 
1 2 -> 
0.3200419999999973
0.3187413215637207


SystemExit: 1

/home/camcomia/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [63]:
#UCS

search([1,2,5,10,12,17,24,21,20,20,11,33,15,19,55],0)

Input:  1 2 5 10 11 12 15 17 19 20 20 21 24 33 55 
Output:  182 50581
1 2 -> 
1 <- 
10 11 -> 
2 <- 
1 2 -> 
1 <- 
12 15 -> 
2 <- 
1 2 -> 
1 <- 
17 19 -> 
2 <- 
1 2 -> 
1 <- 
20 20 -> 
2 <- 
1 2 -> 
1 <- 
21 24 -> 
2 <- 
1 2 -> 
1 <- 
33 55 -> 
2 <- 
1 2 -> 
1 <- 
1 5 -> 
10.907847999999998
10.906993627548218


SystemExit: 1

/home/camcomia/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [64]:
#A*

search([1,2,5,10,12,17,24,21,20,20,11,33,15,19,55],1)

Input:  1 2 5 10 11 12 15 17 19 20 20 21 24 33 55 
Output:  182 49705
1 2 -> 
1 <- 
10 11 -> 
2 <- 
1 2 -> 
1 <- 
12 15 -> 
2 <- 
1 2 -> 
1 <- 
17 19 -> 
2 <- 
1 2 -> 
1 <- 
20 20 -> 
2 <- 
1 2 -> 
1 <- 
33 55 -> 
2 <- 
1 2 -> 
1 <- 
1 5 -> 
1 <- 
21 24 -> 
2 <- 
1 2 -> 
13.851675999999998
13.84962773323059


SystemExit: 1

/home/camcomia/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Analysis and Conclusion

In [1]:
import pandas as pd

labels = ['Algorithm','Runtime(n=4)','Runtime(n=10)','Runtime(n=15)','Optimal?']
data = [('Uniform Cost Search',0.006168999999999869,0.16934900000000042,10.907847999999998,'Yes'),('A* Search',0.004996999999999474,0.3200419999999973,13.851675999999998,'Yes')]
runtime_data = pd.DataFrame.from_records(data, columns = labels)
runtime_data

,Algorithm,Runtime(n=4),Runtime(n=10),Runtime(n=15),Optimal?
0,Uniform Cost Search,0.006169,0.169349,10.907848,Yes
1,A* Search,0.004997,0.320042,13.851676,Yes


In [8]:
data2 = [('Algorithm',['Uniform Cost Search', 'A* Search']),('Nodes (n=4)',[73,66]),('Nodes (n=10)',[2876,2743]),('Nodes (n=15)',[50851,49705])]
nodes_data = pd.DataFrame.from_items(data2)
nodes_data

,Algorithm,Nodes (n=4),Nodes (n=10),Nodes (n=15)
0,Uniform Cost Search,73,2876,50851
1,A* Search,66,2743,49705


For smaller cases, A star search has a better chance of being faster than the uniform cost search. However, for bigger cases, there is a better chance of UCS being faster than A star search. From this we cannot really assume that using one over the other is better. It is better to know your input cases to ensure that you cater the algorithm you use to your data to get the more efficient outcome. 


It can be read from the table that both search algorithms are optimal. In general cases, the deciding factor as to which is the better algorithm is from the nodes expanded. Since UCS is a blind search, it does a worse job than A * search in evaluating the fitness of every prospective state. We can see this comparison take place in the number of nodes expanded by both algorithms in the second table above. We've also learned that, with a good heuristic, A *  search can find the optimal solution with lesser nodes expanded and the only con is that it takes a significantly longer time for larger cases.

# Individual Contributions

**Comia, Camille<br>
Pangarungan, Nicko**

Nicko designed the implementation and the program while Camille made some very minor revisions on it as well. For the written report, since Nicko created the program, he also wrote the description of the program environment and implementation. Camille proved the heuristics for A * search and the rest of the written report.